# M3.4 - Creating a Basin-Scale Water Budget

*Part of:* [**Open Science for Water Resources**](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources)

In [ ]:
import glob
import datetime
import earthaccess
import numpy as np
import h5py
import xarray as xr
import rasterio as rio
from matplotlib import pyplot

auth = earthaccess.login()

$$
P = E + R + \Delta S
$$

![](./assets/water_budget.png)

[*Image courtesy of the USGS*](https://www.usgs.gov/media/images/components-a-simple-water-budget-part-a-watershed)

--- 

## Estimating evapotranspiration for our basin

In [ ]:
import geopandas

basin = geopandas.read_file('/home/arthur.endsley/Workspace/NTSG/projects/Y2024_TOPS_Training/data/YellowstoneRiver_drainage_WSG84.shp')

In [ ]:
basin.bounds.to_numpy().ravel()

In [ ]:
tuple(basin.bounds.to_numpy().ravel())

In [ ]:
# https://dx.doi.org/10.5067/MODIS/MOD16A3GF.061

results = earthaccess.search_data(
    short_name = 'MOD16A3GF',
    temporal = ('2014-01-01', '2023-12-31'),
    bounding_box = tuple(basin.bounds.to_numpy().ravel()))

In [ ]:
results_clean = list(filter(lambda granule: 'h10v04' in granule['meta']['native-id'], results))

In [ ]:
earthaccess.download(results_clean, 'data/MOD16A3')

### Installing a new Python package

In [ ]:
!pip install --upgrade py4eos

In [ ]:
import py4eos

hdf = py4eos.read_hdf4eos('data/MOD16A3/MOD16A3GF.A2014001.h10v04.061.2022077145846.hdf')
hdf

In [ ]:
et = hdf.get('ET_500m')
et

In [ ]:
et = hdf.get('ET_500m', scale_and_offset = True)
et

In [ ]:
ds_et = hdf.to_rasterio('ET_500m', filename = '', driver = 'MEM', scale_and_offset = True)
ds_et

### Compiling an ET time series for our basin

In [ ]:
basin_albers = basin.to_crs(epsg = 5070)
basin_albers

In [ ]:
file_list = glob.glob('data/MOD16A3/*.hdf')
file_list

In [ ]:
gtiff_file_list = []

for filename in file_list:
    # Extract the year from the filename
    year = filename.split('.')[1].replace('A', '').replace('001', '')
    
    # Read in the MODIS MOD16 data
    hdf = py4eos.read_hdf4eos(filename)
    
    # Create a rasterio Dataset for this raster; also write out a GeoTIFF file
    output_filename = f'processed/MOD16A3GF_{year}_ET_500m.tiff'
    ds_et = hdf.to_rasterio(
        'ET_500m', filename = output_filename, scale_and_offset = True)
    ds_et.close()
    
    # Save the output GeoTIFF filename
    gtiff_file_list.append(output_filename)

In [ ]:
gtiff_file_list.sort()
gtiff_file_list

In [ ]:
import rioxarray

ds_et = rioxarray.open_rasterio(gtiff_file_list[0])
ds_et_albers = ds_et.rio.reproject(basin_albers.crs)
ds_et_albers.rio.clip(basin_albers.geometry.values).plot()

In [ ]:
et_series = []

for filename in gtiff_file_list:
    ds_et = rioxarray.open_rasterio(filename)
    ds_et_albers = ds_et.rio.reproject(basin_albers.crs)
    ds_et_basin = ds_et_albers.rio.clip(basin_albers.geometry.values)
    # Again, we take the mean because it is equivalent to spatially
    #    distributing the columns of water in each pixel
    et_series.append(float(ds_et_basin.mean().values))

et_series = np.array(et_series)
et_series

---

## Obtaining basin-scale runoff data

https://www.nature.com/articles/s41597-020-00583-2

In [ ]:
import xarray as xr

ds = xr.open_dataset('/home/arthur/Workspace/NTSG/projects/Y2024_TOPS_Training/data/HYSETS-2023_watershed_YellowstoneRiver.nc')
ds

In [ ]:
ds_10years = ds.sel(time = slice('2014-01-01', '2023-12-31'))
ds_10years

In [ ]:
ds_10years['discharge'].plot()

In [ ]:
# Compute total daily discharge based on the mean rate
ds_10years['discharge_total'] = ds_10years.discharge * 60 * 60 * 24

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

# Compute annual discharge
ds_10years['discharge_total'].resample(time = 'YS').sum().to_series().plot.bar()

## Computing basin-scale runoff

### Calculating basin area

In [ ]:
basin.crs

https://epsg.io/5070

In [ ]:
basin.to_crs(epsg = 5070).crs

In [ ]:
basin_area = basin.to_crs(epsg = 5070).area.values

# Convert from [m] to [km]
basin_area / 1e6

In [ ]:
1.789e11 / 1e6

In [ ]:
ds_10years.drainage_area

In [ ]:
# Get the [m year-1] that this basin drained through Yellowstone River,
#  then convert from [m year-1] to [mm year-1]
runoff_meters_per_yr = ds_10years['discharge_total'].resample(time = 'YS').sum() / basin_area
runoff_mm_per_yr = runoff_meters_per_yr * 1000

runoff_series = runoff_mm_per_yr.values
runoff_series

---

## Putting It All Together

$$
P = E + R + \Delta S \quad\rightarrow\quad \Delta S = P - (E + R)
$$

In [ ]:
ds_precip = xr.open_dataset('processed/IMERG-Final_precip_monthly_2014-2023.nc')

precip_series = ds_precip.precip_monthly.mean(['lon','lat']).groupby('time.year').sum().values
precip_series

In [ ]:
et_series

In [ ]:
runoff_series

In [ ]:
years = np.arange(2014, 2024)
pyplot.plot(years, precip_series, 'b', label = 'Precipitation')
pyplot.plot(years, et_series, 'g', label = 'ET')
pyplot.plot(years, runoff_series, 'r', label = 'Runoff')
pyplot.ylabel('Water Flux (mm per year)')
pyplot.legend()
pyplot.show()

In [ ]:
delta_storage = precip_series - (et_series + runoff_mm_per_yr.values)
delta_storage

In [ ]:
pyplot.bar(years, height = delta_storage)